# Scoring client project

> Ref. Simplonline : https://simplonline.co/briefs/a1e3b434-ff7f-477c-9354-e052fd1a85f7

In [2]:
from IPython.display import display

import os
import pandas as pd

In [3]:
%%time

# Load provided CSV files and stores them in variables such as :
# > df_[<name_of_csv_file_withtout_extension>] = csv data as pd.DataFrame

DISPlAY = False

EXCLUDE = ['HomeCredit_columns_description.csv']
ENCODING = 'ISO-8859-1'
DIR_PATH = './data'
DATA_PATH = os.listdir(DIR_PATH)
N_DATA_CSV = len([f for f in DATA_PATH if f not in EXCLUDE])

list_labels = []
print('Data importation :\n')
for file in DATA_PATH:
    print('-' * 20, '\nFrom :', file)
    if file not in EXCLUDE:
        df = pd.read_csv(os.path.join(DIR_PATH, file), encoding = ENCODING, memory_map = True)
        file_str = file.split('.')[0]
        var_name = 'df_{}'.format(file_str)
        globals()[var_name] = df
        describe = df.describe()
        list_labels.append(df.columns.to_list())
        print('File loaded -> {}'.format(var_name))
    else:
        print('This file is ignored')
    if DISPlAY:
        display(df.head(), describe)
    
if DISPlAY:
    print('\nList of all labels :\n', ', '.join([j for sub in list_labels for j in sub]))
else:
    print('\n\nImport data : OK')

Data importation :

-------------------- 
From : application_test.csv
File loaded -> df_application_test
-------------------- 
From : HomeCredit_columns_description.csv
This file is ignored
-------------------- 
From : POS_CASH_balance.csv
File loaded -> df_POS_CASH_balance
-------------------- 
From : credit_card_balance.csv
File loaded -> df_credit_card_balance
-------------------- 
From : installments_payments.csv
File loaded -> df_installments_payments
-------------------- 
From : application_train.csv
File loaded -> df_application_train
-------------------- 
From : bureau.csv
File loaded -> df_bureau
-------------------- 
From : previous_application.csv
File loaded -> df_previous_application
-------------------- 
From : bureau_balance.csv
File loaded -> df_bureau_balance
-------------------- 
From : sample_submission.csv
File loaded -> df_sample_submission


Import data : OK
CPU times: user 1min 9s, sys: 15.9 s, total: 1min 25s
Wall time: 1min 47s


In [80]:
# Make sure all CSV files were loaded
EXCLUDE_TEST = 'df_application_test'
dfs = {key: globals()[key] for key in globals() if str(key).startswith('df_') and key not in EXCLUDE_TEST}
print('Loaded dataframes ({}/{}) :'.format(len(dfs.keys()), N_DATA_CSV), ', '.join(list(dfs.keys())))

dfs_raw = [globals()[df] for df in list(dfs.keys())]
data_points = []
for df in dfs_raw:
    data_points.append(len(df.columns) * len(df.index))
print('The entiere dataset contains :', sum(data_points), 'data points')

Loaded dataframes (8/9) : df_POS_CASH_balance, df_credit_card_balance, df_installments_payments, df_application_train, df_bureau, df_previous_application, df_bureau_balance, df_sample_submission
The entiere dataset contains : 487672047 data points


> There is a total of 10 CSV files. 9 of those are proper data, one is a description file (data dictionnary like). <br>
> This represents around 2.6GB of disk storage.

In [20]:
#df_full = pd.concat([globals()[df] for df in list(dfs.keys())], axis=1).reset_index()
#df_full = pd.concat(dfs, axis=1).reset_index()

In [ ]:
LABEL_ID = 'SK_ID_PREV'

for i, df in enumerate(dfs):
    print(i, LABEL_ID in df.columns)

In [54]:
info = pd.read_csv('./data/HomeCredit_columns_description.csv', encoding=ENCODING).set_index('Unnamed: 0')

In [99]:
for x in dfs:
    print(x, [y for y in globals()[x].columns if y in df_application_train.columns and x != 'df_application_train'], '\n')

df_POS_CASH_balance ['SK_ID_CURR'] 

df_credit_card_balance ['SK_ID_CURR'] 

df_installments_payments ['SK_ID_CURR'] 

df_application_train [] 

df_bureau ['SK_ID_CURR', 'AMT_ANNUITY'] 

df_previous_application ['SK_ID_CURR', 'NAME_CONTRACT_TYPE', 'AMT_ANNUITY', 'AMT_CREDIT', 'AMT_GOODS_PRICE', 'WEEKDAY_APPR_PROCESS_START', 'HOUR_APPR_PROCESS_START', 'NAME_TYPE_SUITE'] 

df_bureau_balance [] 

df_sample_submission ['SK_ID_CURR', 'TARGET'] 



----

In [132]:
from tpot import TPOTRegressor

In [172]:
target_label = 'TARGET'
exclude_label = [target_label, 'SK_ID_CURR']

# Remove nas
df_application_train = df_application_train.dropna()

X_train = df_application_train[[x for x in df_application_train.columns if x not in exclude_label]]
X_train = pd.get_dummies(X_train)
y_train = df_application_train[target_label]

X_test = df_application_test
X_test = pd.get_dummies(X_test)
y_test = df_sample_submission[target_label]

In [ ]:
reg = TPOTRegressor(verbosity=2, config_dict='TPOT light')
reg.fit(X_train, y_train)